In [1]:
from itertools import product
import numpy as np
import pandas as pd
from dimod import Integer
from dimod import quicksum 
from dimod import ConstrainedQuadraticModel, DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler, LeapHybridCQMSampler 

In [2]:
stocks=('AAPL', 'MSFT', 'AAL', 'WMT') 
budget=1000
file_path='C:/Users/amrit/Desktop/Python/Quantum/PO/dataset/stock_details.csv'
alpha=0.005 
gamma = 100
precision = 2
bin_size = 10

In [3]:
df = pd.read_csv(file_path,index_col=0)

In [4]:
print(df.columns.tolist())

['AAPL', 'MSFT', 'AAL', 'WMT']


In [5]:
max_num_shares = (budget/df.iloc[-1]).astype(int)

In [6]:
print(max_num_shares)

AAPL     85
MSFT     48
AAL     224
WMT      21
Name: November-11, dtype: int32


In [7]:
shares_intervals = {}

In [8]:
for stock in stocks:
            if max_num_shares[stock]+1 <= bin_size:
                shares_intervals[stock] = list(range(max_num_shares[stock] + 1))
            else:
                span = (max_num_shares[stock]+1) / bin_size
                shares_intervals[stock] = [int(i*span) 
                                        for i in range(bin_size)]

In [9]:
shares_intervals

{'AAPL': [0, 8, 17, 25, 34, 43, 51, 60, 68, 77],
 'MSFT': [0, 4, 9, 14, 19, 24, 29, 34, 39, 44],
 'AAL': [0, 22, 45, 67, 90, 112, 135, 157, 180, 202],
 'WMT': [0, 2, 4, 6, 8, 11, 13, 15, 17, 19]}

In [10]:
price = df.iloc[-1]
monthly_returns = df[list(stocks)].pct_change().iloc[1:]
avg_monthly_returns = monthly_returns.mean(axis=0)
covariance_matrix = monthly_returns.cov()

In [11]:
dqm = DiscreteQuadraticModel() 

In [12]:
for s in stocks:
            dqm.add_variable(len(shares_intervals[s]), label=s)

In [13]:
for s1, s2 in product(stocks, stocks):
            coeff = (covariance_matrix[s1][s2]
                        * price[s1] * price[s2])
            if s1 == s2:
                for k in range(dqm.num_cases(s1)):
                    num_s1 = shares_intervals[s1][k]
                    dqm.set_linear_case(
                                    s1, k, 
                                    dqm.get_linear_case(s1,k) 
                                    + alpha*coeff*num_s1*num_s1)
            else:
                for k in range(dqm.num_cases(s1)):
                    for m in range(dqm.num_cases(s2)):
                        num_s1 = shares_intervals[s1][k]
                        num_s2 = shares_intervals[s2][m]

                        dqm.set_quadratic_case(
                                s1, k, s2, m, 
                                dqm.get_quadratic_case(s1,k,s2,m)
                                + coeff*alpha*num_s1*num_s2) 

In [14]:
for s in stocks:
            for j in range(dqm.num_cases(s)):
                dqm.set_linear_case(
                                s, j, dqm.get_linear_case(s,j)
                                - shares_intervals[s][j]*price[s]
                                * avg_monthly_returns[s])

In [15]:
sampler = LeapHybridDQMSampler(token="DEV-e9d7e9eee75c252915b50dfd7220fe54d848586c")

In [16]:
factor = 10**precision
min_budget = round(factor*budget)
budget = int(budget)

terms = [(s, j, int(shares_intervals[s][j]
                *factor*price[s])) 
                for s in stocks 
                for j in range(dqm.num_cases(s))]

dqm.add_linear_inequality_constraint(terms, constant=0, 
                                            lb=min_budget, 
                                            ub=factor*budget, 
                                            lagrange_multiplier=gamma, 
                                            label="budget")

model_DQM = dqm 

In [17]:
dqm = sampler.sample_dqm(model_DQM,label="Example - Portfolio Optimization")

In [18]:
print(dqm)

SolverFailureError: Problem not accepted because user amritrajeev42@gmail.com has insufficient remaining solver access time in project DEV

In [ ]:
def compute_risk_and_returns(solution):
        """Compute the risk and return values of solution.
        """
        variance = 0.0
        for s1, s2 in product(solution, solution):
            variance += (solution[s1] * price[s1] 
                        * solution[s2] * price[s2]  
                        * covariance_matrix[s1][s2])

        est_return = 0
        for stock in solution:
            est_return += solution[stock]*price[stock]*avg_monthly_returns[stock]

        return round(est_return, 2), round(variance, 2)

In [ ]:
solution = {}

sample = dqm.first.sample 
solution['stocks'] = {s:shares_intervals[s][sample[s]] for s in stocks}
        
solution['return'], solution['risk'] = compute_risk_and_returns(solution['stocks'])

spending = sum([price[s]*solution['stocks'][s] for s in stocks])

print(f'\nDQM -- solution for alpha == {alpha} and gamma == {gamma}:')
print(f"\nShares to buy:")

print("\n".join("{}\t{:>3}".format(k, v) for k, v in solution['stocks'].items())) 

print(f"\nEstimated returns: {solution['return']}")

print(f"Purchase Cost: {spending:.2f}")

print(f"Variance: {solution['risk']}\n")